##Tuning with Keras-Tuner
There are variety of libraries that support various hyperparameter optimization methods, we implement using keras-tuner library which is part of **Keras library.** Some other popular libraries to check out are **RayTune(part of Ray)** and **Optuna**, I will add some resources that explain these libraries.

#Parameter: values that are learned by the model and cannot be tuned by Data Scientists. Includes mean, variance of data, weights, bias, etc…

#Hyperparameter: Set beforehand by Data Scientists, it cannot be learned by the model such as number of layers , number of neurons, learning_rate, etc…

Hyperparameter Optimization techniques
This will be a brief conceptual explanation without mathematical details.

#GridSearch

Search space is Cartesian product of all possible hyperparameters.

For example if we want to try out two hyperparameters say learning_rate and optimizer, [0.1, 0.001, 0.0001] and [‘adam’, ‘SGD’] repectively grid search trains model with (0.1, ‘adam’), (0.001, ‘adam’), (0.0001, ‘adam’), (0.1, ‘SGD’), (0.001, ‘SGD’), and (0.0001, ‘SGD) which is all possible variations(cartesian product of two list).

For simple model with small search space GridSearch is actually a pretty good choice since training will take shorter time and all configurations can be covered in reasonable amount of time and resource. However when search space is large and model is complex it isn’t preferred especially when some hyperparameters are more important than others. Looking at picture below we can see that for Gridsearch we can only search 3 different values for both hyperparameters where instead we should’ve searched more different values of important hyperparameter.


#RandomSearch

Randomly search from possible hyperparameter combination. You just need to specify lower and upper bound and from there it is random searched (each value have uniform distribution).

Random and Grid search are both intuitive and simple to implement however they have few downsides:

Need to train model until the end.
Does not make use of previously searched hyperparameter configuration to suggest next hyperparameter configuration to try.

#Hyperband

Hyperband overcome first downside of grid/random search method by Successive halving which is dropping half of worst performing search space after each trial(can be epoch, specified time, etc…), this continues until all budget is used.

Successive halving suffers from budget-vs-number of config trade-off:

try many configurations → small budget for each, split across many. This is a problem when there are slow learners (configurations that perform bad at first however well overall) because each hyperparameter combination is not trained until the end.
try small configurations → larger budget for each however small search space.
Hyperband overcomes this trade-off by dividing total budget into several combinations of number of config vs. budget (some set try large configurations, some try small configuration) then use successive halving on each set.

#BayesianOptimization

Uses past evaluation results to guide which hyperparameter to search next. Its working process is as follows:

Build probabilistic model(surrogate model) of the objective function(hyperparameter that minimize objective, say RMSE).
Try different hyperparameters on probabilistic model and choose the best one.
apply hyperparameter chosen from 2nd step to true objective function.
update probabilistic model incorporating new results
repeat steps 1~4.
#BOHB

Combination of BayesianOptimization and Hyperband, it overcomes both downside of grid/random search. Paper can be found on reference section of this blog.



In [2]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [21]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [23]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values

X.shape, y.shape

((768, 8), (768, 1))

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
X = scaler.fit_transform(X)
X, X.shape

(array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
          0.46849198,  1.4259954 ],
        [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
         -0.36506078, -0.19067191],
        [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
          0.60439732, -0.10558415],
        ...,
        [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
         -0.68519336, -0.27575966],
        [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
         -0.37110101,  1.17073215],
        [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
         -0.47378505, -0.87137393]]),
 (768, 8))

In [26]:
from sklearn.model_selection import   train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [27]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((614, 8), (614, 1), (154, 8), (154, 1))

#With the help of tf.data.Dataset.from_tensor_slices() method, we can get the slices of an array in the form of objects by using tf.data.Dataset.from_tensor_slices() method.

Syntax : tf.data.Dataset.from_tensor_slices(list)

Return : Return the objects of sliced elements.

In [28]:
inputs = tf.keras.Input(shape=(8), batch_size=32)
flatten_layer = tf.keras.layers.Flatten()(inputs)
dense = tf.keras.layers.Dense(units=512, activation='relu')(flatten_layer)
outputs = tf.keras.layers.Dense(units=10)(dense)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [29]:
history = model.fit(X_train,y_train,batch_size=32,epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.7357 - accuracy: 0.7020 - val_loss: 0.4974 - val_accuracy: 0.8052
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5095 - accuracy: 0.7508 - val_loss: 0.4775 - val_accuracy: 0.7922
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4804 - accuracy: 0.7638 - val_loss: 0.4899 - val_accuracy: 0.7792
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4639 - accuracy: 0.7671 - val_loss: 0.4635 - val_accuracy: 0.8052
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4437 - accuracy: 0.7866 - val_loss: 0.4725 - val_accuracy: 0.8117
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4537 - accuracy: 0.7834 - val_loss: 0.4827 - val_accuracy: 0.7922
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4393 - accuracy: 0.7834 - val_loss: 0.4974 - val_accuracy: 0.7792
Epoch 8/100


#Here are hyperparameters to tune:

number of layers

number of neurons per layer

dropout rate

activation function

learning rate

#Two main parts:--------------------->

**build_model:**

build model like above however built with different set of hyperparameter that are chosen by the tuner.

**Tuner:**

class that manage hyperparameter search process, all different HPO methods **RandomSearch, HyperBand, and BayesianOptimization** are subclass of Tuner class.

Below is build model part of our code. We will explain line by line what’s going on.

##Note: hpis parameter that is passed by tuner class at each trial.

lines 2~7 :

 initialize our hyperparameter search space within our function.

9~11:

 hp.Choice() selects value among list of values and hp.Float() chooses from min_value and max_value and depending on ‘sampling’ parameter it values are sampled differently. In our case since we chose ‘log’ equal probabilities are given to each order of magnitude 0.0001, 0.001, and 0.1.

17~18:

 hp.Int() simply choose integer value between range, note that both upper and lower range are inclusive. Here we choose number of neurons to be between 32~512.

19~21:

number of layers to use are chosen by hp.Choice, -1 is because we’ve added one dense layer above. Now for each layer we choose number of neurons to be between 32~512. If you want all layers to have same number of neurons simply use dnn_units from line 18 at each iteration(delete line 20).

22:

hp.Boolean() chooses from True/False, if True it adds dropout layer with 0.25 dropout ratio.


#hp.Int(
    "n_layers",
    min_value=6,
    max_value=12)


The possible values are [6, 7, 8, 9, 10, 11, 12].

In [93]:
def build_model(hp):
    dnn_layers_ss = [1,2,3]
    dnn_units_min, dnn_units_max = 32, 512
    dropout_ss = [0.1, 0.2]
    active_func_ss = ['relu', 'tanh','sigmoid']
    optimizer_ss = ['adam']
    lr_min, lr_max = 1e-4, 1e-1

    active_func = hp.Choice('activation', active_func_ss)
    optimizer = hp.Choice('optimizer', optimizer_ss)
    lr = hp.Float('learning_rate', min_value=lr_min, max_value=lr_max, sampling='log')

    inputs = tf.keras.Input(shape=(8))
    flatten_layer = tf.keras.layers.Flatten()(inputs)

    # create hidden layers
    dnn_units = hp.Int(f"0_units", min_value=dnn_units_min, max_value=dnn_units_max) #0_units
    dense = tf.keras.layers.Dense(units=dnn_units, activation=active_func)(flatten_layer)

    for layer_i in range(hp.Choice("n_layers", dnn_layers_ss) - 1 ):
        dnn_units = hp.Int(f"{layer_i}_units", min_value=dnn_units_min, max_value=dnn_units_max)
        dense = tf.keras.layers.Dense(units=dnn_units, activation=active_func)(dense)
        if hp.Boolean("dropout"):
            dense = tf.keras.layers.Dropout(rate=0.25)(dense)


    outputs = tf.keras.layers.Dense(units=10)(dense)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "SGD":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    else:
        raise("Not supported optimizer")

    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [64]:
tuner = kt.RandomSearch(hypermodel=build_model, objective='val_accuracy',max_trials=35,directory="mydir",project_name="samim13")

Reloading Tuner from mydir/samim13/tuner0.json


In [65]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 35 Complete [00h 00m 03s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 58m 34s


In [66]:
tuner.get_best_hyperparameters()[0].values

{'activation': 'relu',
 'optimizer': 'adam',
 'learning_rate': 0.0022949470996957142,
 '0_units': 463,
 'n_layers': 2,
 'dropout': False}

In [67]:
model = tuner.get_best_models(num_models=1)[0]

In [68]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 flatten (Flatten)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 463)               4167      
                                                                 
 dense_1 (Dense)             (None, 463)               214832    
                                                                 
 dense_2 (Dense)             (None, 10)                4640      
                                                                 
Total params: 223639 (873.59 KB)
Trainable params: 223639 (873.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
history = model.fit(X_train,y_train,batch_size=32, epochs=20, validation_data=(X_test,y_test))

Epoch 1/20
20/20 [==============================] - 0s 11ms/step - loss: 0.0241 - accuracy: 0.9935 - val_loss: 2.1322 - val_accuracy: 0.7078
Epoch 2/20
20/20 [==============================] - 0s 7ms/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 2.1753 - val_accuracy: 0.7078
Epoch 3/20
20/20 [==============================] - 0s 8ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 2.1886 - val_accuracy: 0.7273
Epoch 4/20
20/20 [==============================] - 0s 8ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 2.2136 - val_accuracy: 0.7078
Epoch 5/20
20/20 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 0.9984 - val_loss: 2.2623 - val_accuracy: 0.7208
Epoch 6/20
20/20 [==============================] - 0s 9ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 2.3014 - val_accuracy: 0.7338
Epoch 7/20
20/20 [==============================] - 0s 8ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 2.3160 - val_accuracy: 0.7143
Epoch 8/20
20/20 [=

#After building tuner with our model, we run .search() method to search our hyperparameter search space. Note that at each trial our tuner sends different hyperparameters to train with, each tuner subclass(RandomSearch, Hyperband, BO) will use their own strategies on choosing next hyperparameters.

In [75]:
def build_tuner(model, hpo_method, objective, dir_name):
    if hpo_method == "RandomSearch":
        tuner = kt.RandomSearch(model, objective=objective, max_trials=3, executions_per_trial=1,
                               project_name=hpo_method, directory=dir_name)
    elif hpo_method == "Hyperband":
        tuner = kt.Hyperband(model, objective=objective, max_epochs=3, executions_per_trial=1,
                            project_name=hpo_method)
    elif hpo_method == "BayesianOptimization":
        tuner = kt.BayesianOptimization(model, objective=objective, max_trials=3, executions_per_trial=1,
                                       project_name=hpo_method)
    return tuner



In [82]:
obj = kt.Objective('val_accuracy', direction='max')
dir_name = "v1"
randomsearch_tuner = build_tuner(build_model, "RandomSearch", obj, dir_name)
randomsearch_tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Reloading Tuner from v1/RandomSearch/tuner0.json


In [80]:
model = build_model(randomsearch_tuner.get_best_hyperparameters(1)[0])
model.fit(X_train,y_train,batch_size=32, epochs=150, validation_data=(X_test,y_test))

Epoch 1/150
20/20 [==============================] - 1s 21ms/step - loss: 1.8039 - accuracy: 0.5945 - val_loss: 1.4645 - val_accuracy: 0.7662
Epoch 2/150
20/20 [==============================] - 0s 11ms/step - loss: 1.4138 - accuracy: 0.7378 - val_loss: 1.1908 - val_accuracy: 0.8052
Epoch 3/150
20/20 [==============================] - 0s 12ms/step - loss: 1.0946 - accuracy: 0.7459 - val_loss: 0.8292 - val_accuracy: 0.7792
Epoch 4/150
20/20 [==============================] - 0s 13ms/step - loss: 0.7184 - accuracy: 0.7606 - val_loss: 0.5905 - val_accuracy: 0.7922
Epoch 5/150
20/20 [==============================] - 0s 12ms/step - loss: 0.5543 - accuracy: 0.7427 - val_loss: 0.5103 - val_accuracy: 0.7857
Epoch 6/150
20/20 [==============================] - 0s 19ms/step - loss: 0.5191 - accuracy: 0.7622 - val_loss: 0.4871 - val_accuracy: 0.7727
Epoch 7/150
20/20 [==============================] - 0s 19ms/step - loss: 0.5066 - accuracy: 0.7606 - val_loss: 0.4823 - val_accuracy: 0.7727
Epoch 

In [84]:
obj = kt.Objective('val_accuracy', direction='max')
dir_name = "v1"
hyperband_tuner = build_tuner(build_model, "Hyperband", obj, dir_name)
hyperband_tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Reloading Tuner from ./Hyperband/tuner0.json


In [86]:
model = build_model(hyperband_tuner.get_best_hyperparameters(1)[0])
model.fit(X_train,y_train,batch_size=32, epochs=50, validation_data=(X_test,y_test))

Epoch 1/50
20/20 [==============================] - 1s 12ms/step - loss: 1.4590 - accuracy: 0.6336 - val_loss: 0.7578 - val_accuracy: 0.8117
Epoch 2/50
20/20 [==============================] - 0s 5ms/step - loss: 0.6427 - accuracy: 0.7704 - val_loss: 0.5205 - val_accuracy: 0.7857
Epoch 3/50
20/20 [==============================] - 0s 4ms/step - loss: 0.5134 - accuracy: 0.7524 - val_loss: 0.4926 - val_accuracy: 0.7857
Epoch 4/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4951 - accuracy: 0.7541 - val_loss: 0.4727 - val_accuracy: 0.7922
Epoch 5/50
20/20 [==============================] - 0s 7ms/step - loss: 0.4821 - accuracy: 0.7720 - val_loss: 0.4978 - val_accuracy: 0.7727
Epoch 6/50
20/20 [==============================] - 0s 5ms/step - loss: 0.5092 - accuracy: 0.7524 - val_loss: 0.4747 - val_accuracy: 0.7857
Epoch 7/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4818 - accuracy: 0.7818 - val_loss: 0.4827 - val_accuracy: 0.7662
Epoch 8/50
20/20 [=

In [94]:
obj = kt.Objective('val_accuracy', direction='max')
dir_name = "v1"
BayesianOptimization_tuner = build_tuner(build_model,"BayesianOptimization", obj, dir_name)
BayesianOptimization_tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Reloading Tuner from ./BayesianOptimization/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation
adamax            |adamax            |optimizer
0.0020582         |0.0020582         |learning_rate
443               |443               |0_units
3                 |3                 |n_layers
True              |True              |dropout



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 164, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 155, in _build_hypermodel
    model

RuntimeError: ignored

In [92]:
model = build_model(BayesianOptimization_tuner.get_best_hyperparameters(1)[0])
model.fit(X_train,y_train,batch_size=32, epochs=50, validation_data=(X_test,y_test))

TypeError: ignored